### ***Problem01***
---

In [ ]:
### Import library & data set
# library
from google.colab import drive
drive.mount("/content/drive")
import re
import numpy as np
import pandas as pd
from itertools import pairwise
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# dataset
path = "/content/drive/MyDrive/DS5230_Unsupervised Machine Learning/Assignments/HW06/u.data.csv"
df = pd.read_csv(path,sep="\t",header=None,index_col=None)
df.drop([3],axis=1,inplace=True)
df = df.rename(columns={0:"user_id",1:"movie_id",2:"ratings"})
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,user_id,movie_id,ratings
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [ ]:
### find movie title
# movie dataset
item_path = "/content/drive/MyDrive/DS5230_Unsupervised Machine Learning/Assignments/HW06/u.item.csv"
df_item = pd.read_csv(item_path,sep="|",header=None,encoding="iso-8859-1")
df_item = df_item[[0,1]]
df_item.rename(columns={0:"movie_id",1:"title"},inplace=True)
df_item.head()

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [ ]:
### get similarity
# pivot & similarity
df_pivot = df.pivot(index="user_id",columns="movie_id",values="ratings")
df_pivot = df_pivot.fillna(0)
user_similarity = pd.DataFrame(cosine_similarity(df_pivot,df_pivot),
                               index=df_pivot.index,columns=df_pivot.index)
user_similarity

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.166931,0.047460,0.064358,0.378475,0.430239,0.440367,0.319072,0.078138,0.376544,...,0.369527,0.119482,0.274876,0.189705,0.197326,0.118095,0.314072,0.148617,0.179508,0.398175
2,0.166931,1.000000,0.110591,0.178121,0.072979,0.245843,0.107328,0.103344,0.161048,0.159862,...,0.156986,0.307942,0.358789,0.424046,0.319889,0.228583,0.226790,0.161485,0.172268,0.105798
3,0.047460,0.110591,1.000000,0.344151,0.021245,0.072415,0.066137,0.083060,0.061040,0.065151,...,0.031875,0.042753,0.163829,0.069038,0.124245,0.026271,0.161890,0.101243,0.133416,0.026556
4,0.064358,0.178121,0.344151,1.000000,0.031804,0.068044,0.091230,0.188060,0.101284,0.060859,...,0.052107,0.036784,0.133115,0.193471,0.146058,0.030138,0.196858,0.152041,0.170086,0.058752
5,0.378475,0.072979,0.021245,0.031804,1.000000,0.237286,0.373600,0.248930,0.056847,0.201427,...,0.338794,0.080580,0.094924,0.079779,0.148607,0.071459,0.239955,0.139595,0.152497,0.313941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.118095,0.228583,0.026271,0.030138,0.071459,0.111852,0.107027,0.095898,0.039852,0.071460,...,0.066039,0.431154,0.258021,0.226449,0.432666,1.000000,0.087687,0.180029,0.043264,0.144250
940,0.314072,0.226790,0.161890,0.196858,0.239955,0.352449,0.329925,0.246883,0.120495,0.342961,...,0.327153,0.107024,0.187536,0.181317,0.175158,0.087687,1.000000,0.145152,0.261376,0.241028
941,0.148617,0.161485,0.101243,0.152041,0.139595,0.144446,0.059993,0.146145,0.143245,0.090305,...,0.046952,0.203301,0.288318,0.234211,0.313400,0.180029,0.145152,1.000000,0.101642,0.095120


In [ ]:
### cf model
"""
build recommendation system
  어떤 function을 만들 것인가?: input으로 user_id, movie_id를 넣으면,
  해당 user와 연관된 user들의 해당 sum(movie_id) / 해당 user_id와 연관된 sum(user_similarity)
  계산하여 해당 값 즉, rating을 도출

input user의 neighbor은 어떻게 설정할 것인가?
"""
# train test split
y = df["user_id"]
X_train,X_test,y_train,y_test = train_test_split(df,y,test_size=0.2,random_state=22)

# rmse
"""estiamte accuracy"""
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# score
"""implement cf_knn and get accuracy"""
def score(model,neighbor_size=10):
  id_pairs = zip(X_test["user_id"],X_test["movie_id"])
  y_pred = np.array([model(user,movie,neighbor_size) for (user,movie) in id_pairs])
  y_true = np.array(X_test["ratings"])
  return rmse(y_true,y_pred)

# knn
def cf_knn(user_id,movie_id,neighbor_size=10):
  """
  build cf via knn
    movie_ratings = movie_id's user,rating
    refining only who watch the movie
  """
  if movie_id in df:
    sim_scores = user_similarity[user_id].copy()
    # movie_ratings = df.loc[movie_id].copy()
    movie_ratings = df[df["movie_id"]==movie_id].copy()
    # drop the null values among users
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.drop(none_rating_idx)
    sim_scores = sim_scores.drop(none_rating_idx)

    """divide as size of neighbor"""
    if neighbor_size==0:
      """calculate movie predict rates"""
      mean_rating = np.dot(sim_scores,movie_ratings)/sim_scores.sum()

    else:
      if len(sim_scores) > 1:
        neighbor_size = min(neighbor_size, len(sim_scores))
        sim_scores = np.array(sim_scores)
        movie_ratings = np.array(movie_ratings)
        user_idx = np.argsort(sim_scores)
        # sim_scores = sim_scores[user_idx][-neighbor_size:] # sim_scores
        sim_scores = user_idx[-neighbor_size:] # ordered ascending orders
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()

      else:
          mean_rating = 3.0
  else:
      mean_rating = 3.0
  return mean_rating

# get accuracy
acc = round(score(cf_knn, neighbor_size=30),2)
print("test set accuracy: {}".format(acc))

test set accuracy: 1.25
